# PS1Ex4 - Discrete Choice

I will once again modify my code from Exercise 2 to fit the discrete-choice problem, following Jason's methods.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Set Parameter Values

Parameters:
$$ 
\beta = \text{ Rate of time preference} \\
b = \text{ Unemployment benefits} \\
\mu = {Mean of log wages} \\
\sigma = {Standard deviation of wage draws
$$

In [2]:
# Declare Parameters
beta = .96
b = .05
mu = 0
sigma = .15

In [ ]:
I also declare some functions:

In [11]:
def util(w):
    '''
    This function defines the lifetime utility that a worker will get 
    if they choose to take a job at wage w.
    '''
    u = w * ( beta / (1 - beta) )
    return u

print(util(2))

47.99999999999996


## Step 2: Create Grid for State Space

In [18]:
'''
------------------------------------------------------------------------
Create grid for possible wages:
------------------------------------------------------------------------
size_w = integer, number of grid points in wage state space
w = 1 dimensional array, possible values for wage offered
ln(w) = N(mu, sigma) 
------------------------------------------------------------------------
'''
size_w = 300

x = np.random.normal(mu, sigma, size_w)
w = np.exp(x)

'''
'''


'''
------------------------------------------------------------------------
Create grid of current utility values    
------------------------------------------------------------------------
U        = 1 dimensional array, lifetime PDV of wages for all possible 
           wage offers (utility if job offer is accepted)
EU       = Expected PDV of all wages in the future - approximated 
           by averaging U for all the wage offers in w (should get 
           us close enough)
------------------------------------------------------------------------
'''
U = util(w)
EU = np.zeros(size_w)
eu = np.sum(U) / len(U)
EU = EU + eu 
# probably woulda been better to use the "islike" numpy here...

## Step 3: VFI

In [ ]:
'''
------------------------------------------------------------------------
Value Function Iteration    
------------------------------------------------------------------------
VFtol       = scalar, tolerance required for value function to converge
VFdist      = scalar, distance between last two value functions
VFmaxiter   = integer, maximum number of iterations for value function
V           = vector, the value functions at each iteration
TV          = vector, the value function after applying the Bellman operator
PF_discrete = vector, indicies of choice of take offer (=0) or wait (=1)
              for all wage offers
Vstore      = array, stores V at each iteration 
VFiter      = integer, current iteration number
Utake       = vector, utility from accepting job now
Uwait       = vector, utility from accepting job tomorrow
VF          = vector, the "true" value function
------------------------------------------------------------------------
'''

VFtol = 1e-6
VFdist = 7.0
VFmaxiter = 3000
V = np.zeros((size_w)) #initial guess at value function
PF_discrete = np.zeros(size_w) # initial guess at policy function
Vmat = np.zeors(size_w)
Vstore = np.zeros((size_w, VFmaxiter)) #initialize Vstore
VFiter = 1
while VFdist > VFtol and VFiter < VFmaxiter:
    for i in range(size_w):
        TV[i] = max(util())